In [ ]:
# Title: Sample Summary & Descriptives
# Author: Anna Zink
# Date: June 21, 2024
# Description: 
#  - Describe the sample 
#  - Describe task sapmples 
#  - Evaluate how access impacts conditions reported in the EHR
# March 10, 2025 - Update to include data summaries for Figure 1

In [ ]:
# load packages 
library(viridis)    # A nice color scheme for plots.
library(ggthemes)   # Common themes to change the look and feel of plots.
library(scales)     # Graphical scales map data to aesthetics in plots.
library(skimr)      # Better summaries of data.
library(lubridate)  # Date library from the tidyverse.
library(tidyverse)  # Data wrangling packages.
library(bigrquery)  # Data extraction from Google BigQuery
library(biglm)      # running lm on big data 
library(survival)
library(broom)
library(janitor)

In [ ]:
load_data<-function(file, folder){
    my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
    system(paste0("gsutil cp ", my_bucket, folder, file, " ."), intern=T)
    dsn <- read_csv(file, show_col_types = FALSE)
    return(dsn)
}

# Replace df with THE NAME OF YOUR DATAFRAME
# folder = "/ehr/" 
write_csv<-function(df, fn, folder) {
   my_dataframe <- df
   destination_filename <- fn
   write_excel_csv(my_dataframe, destination_filename)
   my_bucket <- Sys.getenv('WORKSPACE_BUCKET')
   system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, folder), intern=T)
}

# Read in data and create variables

In [ ]:
# load data and flag people in the analysis samlpe
all<-load_data('all_participant_demo.csv',"/data/")
sample<-load_data('analysis_sample.csv', "/analysis/")
sample$in_sample<-1
all<-merge(all, sample, by='person_id', all.x=TRUE)
all <- clean_names(all)
all[is.na(all)]<-0

In [ ]:
# Derive variables 

# calculate age as of 2023 (last year of enrollment with data)
all$age <- as.integer(floor(interval(all$date_of_birth, ymd("2023-01-01")) / years(1)))
all$age_cat<-ifelse(all$age<25, '18-24',
             ifelse(all$age<45, '25-44',
             ifelse(all$age<64, '45-64',
             ifelse(all$age>=65, '65+', NA))))


# gender
all$male<-ifelse(all$gender == "Male", 1, 0)
all$fem<-ifelse(all$gender == "Female", 1, 0)
all$gender_other<-ifelse(!(all$fem | all$male), 1, 0)

# race and ethnicity
all$white<-ifelse(all$race == "White", 1, 0)
all$black<-ifelse(all$race == "Black or African American", 1, 0)
all$race_unknown<-ifelse(all$race %in% c('None Indicated','MPI: Skip','I prefer not to answer'), 1, 0)
all$asian<-ifelse(all$race == 'Asian', 1, 0)
all$ai_an<-ifelse(all$race == 'American Indian or Alaska Native', 1, 0)
all$race_other<-ifelse(all$race %in% c('More than one population',
                                         'None of these',
                                         'Middle Eastern or North African',
                                        'Native Hawaiian or Other Pacific Islander'), 1, 0)
all$is_latino<-ifelse(all$ethnicity == "Hispanic or Latino", 1, 0)
all$not_hispanic<-ifelse(all$ethnicity == 'Not Hispanic or Latino',1,0)
all$ethnicity_unknown<-ifelse(!(all$ethnicity %in% c("Hispanic or Latino",'Not Hispanic or Latino')), 1, 0)

all$race_eth_cat<-ifelse(all$ethnicity == "Hispanic or Latino", "Hispanic", 
              ifelse(all$race == "American Indian or Alaska Native", "AI/AN",
              ifelse(all$race == "Asian", "Asian", 
              ifelse(all$race == "Black or African American", "Black",
              ifelse(all$race == "White", "White", 
              ifelse(all$race %in% c('More than one population',
                                         'None of these',
                                         'Middle Eastern or North African',
                                        'Native Hawaiian or Other Pacific Islander'), "Other", NA))))))

# insurance
all$ins_uninsured<-ifelse(all$ins_none == 1 | all$ins_indian ==1 | all$anyins_no == 1, 1, 0)
all$ins_medicare<-all$ins_medicare
all$ins_medicaid<-all$ins_medicaid
all$ins_employ<-all$ins_employer_or_union
all$ins_unknown<-ifelse(all$ins_pmi_skip ==1 | all$ins_invalid ==1, 1, 0)
all$ins_other<-ifelse(all$ins_va == 1 | all$ins_military == 1 | 
                      all$ins_purchased == 1 | all$ins_other_health_plan == 1, 1, 0)

all$ins_cat<-ifelse(all$ins_uninsured == 1, 'uninsured',
             ifelse(all$ins_medicare == 1, 'medicare',
             ifelse(all$ins_medicaid == 1, 'medicaid',
             ifelse(all$ins_employ == 1, 'employee',
             ifelse(all$ins_other == 1, 'other', NA)))))

# household income 
all$inc_10<-all$inc_less_10k
all$inc_10_49<-ifelse(all$inc_10k_25k == 1 | all$inc_25k_35k == 1 | all$inc_35k_50k == 1, 1, 0)
all$inc_50_99<-ifelse(all$inc_50k_75k == 1 | all$inc_75k_100k == 1, 1, 0)
all$inc_100_199<-ifelse(all$inc_100k_150k == 1 | all$inc_150k_200k == 1, 1, 0)
all$inc_200<-all$inc_more_200k
all$inc_unknown<-all$inc_skip_unknown

all$inc_cat<-ifelse(all$inc_10 == 1, '<10',
             ifelse(all$inc_10_49 == 1, '10-49k',
             ifelse(all$inc_50_99 == 1, '50-99k',
             ifelse(all$inc_100_199 == 1, '100-199k',
             ifelse(all$inc_200 == 1, '200k+', NA)))))

In [ ]:
# subset to analytic sample and add in a few additional relevant variables 
data<-all[all$in_sample == 1,]
data$noaccess<-ifelse(data$afford_ind | data$delayed_ind, 1, 0)
#data$age <- as.integer(floor(interval(data$date_of_birth, data$current_date) / years(1)))

In [ ]:
## data for flowchart
run<-0
if (run == 1) {
    visits_16_23<-data.frame()
    for (yr in seq(2016, 2023)) {
        print(yr)
        visits<-load_data(paste0('visits_', yr, '.csv'), "/ehr/")
        visits<-merge(visits, data[,c('person_id','noaccess')], by.x='PERSON_ID', by.y='person_id')
        visits<-visits[,c('PERSON_ID','VISIT_START_DATE','VISIT_END_DATE','SRC_ID','VISIT_TYPE','year')]
        visits_16_23<-rbind(visits_16_23, visits)
    }
    dim(visits_16_23)
    write_csv(visits_16_23, 'visits_16_23.csv', '/ehr/')
}

## Panel a. 
- Visit counts
- clinical sites
- % low access

In [ ]:
## data for flowchart
## 134,513 patients with 7,283,987 visits to 50 clinical sites, 53.6% with low access to care 
# merge in visit occurence data
# Pull in visit data from 2018 - 2019 and create list of all participants with a visit 
visits_16_23<-load_data('visits_16_23.csv', '/ehr/')

# GET # OF SITES
length(unique(visits_16_23$SRC_ID))

In [ ]:
# TO GET # OF VISITS NEED TO SORT AND FIND OVERLAPPING VISIT 

visits_16_23<-visits_16_23[order(visits_16_23$PERSON_ID, visits_16_23$SRC_ID, 
                                 visits_16_23$VISIT_START_DATE, visits_16_23$VISIT_END_DATE),]

visits_16_23<-visits_16_23 %>% group_by(PERSON_ID, SRC_ID) %>% mutate(visit_id = row_number(), next_visit=lead(VISIT_START_DATE))

visits_16_23$prev_end<-lag(visits_16_23$VISIT_END_DATE)
visits_16_23$new_encounter<-ifelse(visits_16_23$prev_end + 1 < visits_16_23$VISIT_START_DATE | visits_16_23$visit_id == 1, 1, 0)

# Use new encounter flag to create an encounter ID
visits_16_23<-visits_16_23 %>% group_by(PERSON_ID, SRC_ID) %>% mutate(encounter_id = cumsum(new_encounter))
head(visits_16_23)

In [ ]:
byperson<-visits_16_23 %>% group_by(PERSON_ID, SRC_ID) %>% summarise(visits=n_distinct(encounter_id))
sum(byperson$visits)

In [ ]:
# pct with low access
data$low_access<-ifelse(data$afford_ind ==1 | data$delayed_ind == 1, 1, 0)
prop.table(table(data$low_access))

## Panel b. sample restrictions

- See Analysis Sample.ipynb for exclusion restriction and sample counts.


## Panel c. Self report vs EHR

Create box-plots by access group for # of self-reported conditions and # of EHR recorded conditions.

- See Reliability Analysis.ipynb for code to create these csv

## Panels d-g. demographic distributions

In [ ]:
# age distribution
data$bin_value<-ifelse(data$AGE <= 25, '18 to 25',
                ifelse(data$AGE <= 35, '26 to 35',
                ifelse(data$AGE <= 45, '36 to 45',
                ifelse(data$AGE <= 55, '46 to 55',
                ifelse(data$AGE <= 65, '56 to 65',
                ifelse(data$AGE <= 75, '66 to 75', '76+'))))))
df<-as.data.frame(table(data$bin_value))
names(df)<-c('bin_value','count')
write_csv(df, 'plot_age_distribution.csv', "/output/")

In [ ]:
# race distribution
df<-as.data.frame(table(data$RACE))
names(df)<-c('race','count')
write_csv(df, 'race.csv', "/output/")

In [ ]:
# ethnicity distribution
data$eth<-ifelse(data$ETHNICITY %in% c('Hispanic or Latino', 'Not Hispanic or Latino'), data$ETHNICITY, 'Unknown')
df<-as.data.frame(table(data$eth))
names(df)<-c('ethnicity','count')
write_csv(df, 'ethnicity.csv', "/output/")

In [ ]:
# visit type
visit_dist<-visits_16_23 %>% group_by(PERSON_ID, encounter_id) %>% summarise(visit_type=first(VISIT_TYPE))

In [ ]:
types<-c('Outpatient Visit','Office Visit','Telehealth','Laboratory Visit','Inpatient Visit', 'Pharmacy visit',
        'Ambulatory Rehabilitation Visit','Emergency Room Visit')
visit_dist$type<-ifelse(visit_dist$visit_type %in% types, visit_dist$visit_type, 'Other')
table(visit_dist$type)

In [ ]:
visit_list<-visit_dist %>% group_by(visit_type) %>% summarise(n=n()) %>% arrange(-n)
head(visit_list, n=10)

In [ ]:
df<-as.data.frame(table(visit_dist$type))
names(df)<-c('visit_type','count')
write_csv(df, 'plot_visit_type_distribution.csv', "/output/")

## Time-to-visit cummulative incidence 

In [ ]:
keepvars1<-c('person_id','mindate','afford_ind','delayed_ind')
keepvars2<-c('PERSON_ID','VISIT_START_DATE', 'VISIT_TYPE')
visit_df<-merge(data[,keepvars1], visits_16_23[,keepvars2], by.x='person_id', by.y='PERSON_ID')

In [ ]:
# Convert both to Date
visit_df$index_dt <- as.Date(visit_df$mindate)
visit_df$visit_dt <- as.Date(visit_df$VISIT_START_DATE)

# Calculate difference in days, then convert to months (~30.44 days/month)
visit_df$time_to <- as.numeric(visit_df$visit_dt - visit_df$index_dt) / 30.44

# Round as needed
visit_df$time_to <- round(visit_df$time_to)

In [ ]:
# calculate time to visit 
#visit_df$index_dt<-as.POSIXct(visit_df$mindate, tz = "UTC")
#visit_df$visit_dt<-as.Date(visit_df$VISIT_START_DATE)
#visit_df$time_to<-time_length(interval(visit_df$index_dt, visit_df$visit_dt), 'months')
#visit_df$time_to<-round(visit_df$time_to)

In [ ]:

# flag visits before index and after index and sum
visit_df$ehr_pre<-ifelse(visit_df$time_to<0, 1, 0)
visit_df$ehr_post<-ifelse(visit_df$time_to>=0, 1, 0)

pre_post_summ <- visit_df %>% group_by(person_id) %>% summarise(ehr_visit_pre=max(ehr_pre),
                                           ehr_visit_post=max(ehr_post),
                                           ehr_visit_pre_count=sum(ehr_pre),
                                           ehr_visit_post_count=sum(ehr_post))

In [ ]:
visit_summ<-visit_df %>% filter(time_to >= 0) %>% group_by(person_id)  %>% summarise(time_to=min(time_to))

In [ ]:
sample_list<-data %>% group_by(person_id, noaccess) %>% summarise(n=n())
sample_list<-merge(sample_list, visit_summ, by='person_id', all.x=TRUE)
sample_list$event24<-ifelse(is.na(sample_list$time_to) | sample_list$time_to>24, 0, 1)
sample_list$time_to_24<-ifelse(sample_list$event24 == 0, 24, sample_list$time_to)

In [ ]:
# kaplan-meyer survival estimates 
fit <- survfit(Surv(time_to_24, event24) ~ noaccess, data = sample_list)
df_km<-tidy(fit)

# need to take 1- to report the cummulative incidence 
df_km<-tidy(fit)
df_km <- df_km %>%
  mutate(
    est = 1 - estimate,
    lb = 1 - conf.high,
    ub = 1 - conf.low
  )
head(df_km)

keepvars<-c('time','n.risk','n.event','strata','est','lb','ub')
write_csv(df_km[,keepvars], 'kaplan_meyer_visit_curve.csv', '/output/')

In [ ]:
# raw values - they are the samea s the estimates 
sample_list$time_to_updt<-ifelse(sample_list$event24 == 0, 25, sample_list$time_to)
bymonth<-sample_list %>% group_by(noaccess, time_to_updt) %>% summarise(n=n())
bymonth<-bymonth %>% group_by(noaccess) %>% mutate(total=sum(n))
bymonth$pct<-bymonth$n/bymonth$total
bymonth<- bymonth %>% group_by(noaccess) %>% mutate(cum_pct = cumsum(pct))

# Create Table 1 

In [ ]:
# group by and summarize -- overall, high access, low access (AND), delay, afford
get_summary<-function(df, label){
    summ<-df %>% summarize(delayed=mean(delayed_ind),
                   afford=mean(afford_ind),
                   hc_int_1yr=mean(hc_int_1_yr),
                   hc_int_2yr=mean(hc_int_2_yr),
                   age=mean(age),
                   male=mean(male),
                   fem=mean(fem),
                   gender_other=mean(gender_other),
                   white=mean(white),
                   black=mean(black),
                   asian=mean(asian),
                   ai_an=mean(ai_an),
                   race_unknown=mean(race_unknown),
                   race_other=mean(race_other),
                   is_latino=mean(is_latino),
                   not_hispanic=mean(not_hispanic),
                   eth_unknown=mean(ethnicity_unknown),
                   ins_uninsured = mean(ins_uninsured),
                   ins_medicare = mean(ins_medicare),
                   ins_medicaid = mean(ins_medicaid),
                   ins_employ = mean(ins_employ),
                   ins_unknown = mean(ins_unknown),
                   ins_other = mean(ins_other), 
                   inc_10 = mean(inc_10),
                   inc_10_49 = mean(inc_10_49),
                   inc_50_99 = mean(inc_50_99),
                   inc_100_199 = mean(inc_100_199),
                   inc_200 = mean(inc_200),
                   inc_unknown = mean(inc_unknown),
                   n=n())
    
    summ$group<-label
    return(summ)
}

# group by and summarize -- overall, high access, low access (AND), delay, afford
get_group_summary<-function(df, label, groupvar){
    summ<-df %>% group_by({{ groupvar }}) %>% summarize(delayed=mean(delayed_ind),
                   afford=mean(afford_ind),
                   age=mean(age),
                   male=mean(male),
                   fem=mean(fem),
                   gender_other=mean(gender_other),
                   white=mean(white),
                   black=mean(black),
                   asian=mean(asian),
                   ai_an=mean(ai_an),
                   race_unknown=mean(race_unknown),
                   race_other=mean(race_other),
                   is_latino=mean(is_latino),
                   not_hispanic=mean(not_hispanic),
                   eth_unknown=mean(ethnicity_unknown),
                   ins_uninsured = mean(ins_uninsured),
                   ins_medicare = mean(ins_medicare),
                   ins_medicaid = mean(ins_medicaid),
                   ins_employ = mean(ins_employ),
                   ins_unknown = mean(ins_unknown),
                   ins_other = mean(ins_other), 
                   inc_10 = mean(inc_10),
                   inc_10_49 = mean(inc_10_49),
                   inc_50_99 = mean(inc_50_99),
                   inc_100_199 = mean(inc_100_199),
                   inc_200 = mean(inc_200),
                   inc_unknown = mean(inc_unknown),
                   n=n())
    
    summ$group<-label
    return(summ)
}

In [ ]:
all_of_us<-get_summary(all, 'all of us')

overall<-get_summary(data, 'overall')

data$access<-ifelse(data$afford_ind == 0 & data$delayed_ind == 0, 1, 0)
high_access<-get_summary(data[data$access == 1,], 'High Access')

data$low_access<-ifelse(data$afford_ind == 1 & data$delayed_ind == 1, 1, 0)
low_access<-get_summary(data[data$low_access == 1,], 'Low Access')

# afford
afford<-get_summary(data[data$afford_ind == 1,], 'Affordability')

# delay
delay<-get_summary(data[data$delayed_ind == 1,], 'Delay')

all<-rbind(all_of_us, overall, high_access, low_access, afford, delay)
all

In [ ]:
# folder = "/ehr/" 
write_csv(all, 'table_1_sample_summary.csv', "/data")

In [ ]:
dim(data)

In [ ]:
# 2 x 2 - delay x afford
access_tab<-table(data$delayed_ind, data$afford_ind)
access_tab
prop.table(access_tab)

In [ ]:
prop.table(table(data$afford_ind))

# Create ACS comparison table

remove other for each category (following ACS)

In [ ]:
run_acs_summary<-function(df) {
    # age
    age<-df %>% filter(!is.na(age_cat)) %>% group_by(age_cat) %>% summarise(n=n()) %>%
      mutate(percent = n / sum(n) * 100) %>% rename(cat=age_cat)

    # gender
    gender<-df %>% filter(gender %in% c('Female','Male')) %>% group_by(gender) %>% summarise(n=n()) %>%
      mutate(percent = n / sum(n) * 100) %>% rename(cat=gender)

    # race_ethnicity 
    race<-df %>% filter(!is.na(race_eth_cat)) %>% group_by(race_eth_cat) %>% summarise(n=n()) %>%
      mutate(percent = n / sum(n) * 100) %>% rename(cat=race_eth_cat)

    # ins
    ins<-df %>% filter(!is.na(ins_cat)) %>% group_by(ins_cat) %>% summarise(n=n()) %>%
      mutate(percent = n / sum(n) * 100) %>% rename(cat=ins_cat)

    # income
    income<-df %>% filter(!is.na(inc_cat)) %>% group_by(inc_cat) %>% summarise(n=n()) %>%
      mutate(percent = n / sum(n) * 100) %>% rename(cat=inc_cat)

    #all
    summ<-rbind(age, gender, race, ins, income)
    return(summ)
}

In [ ]:
acs_all<-run_acs_summary(all)
acs_analysis<-run_acs_summary(data)

write_csv(acs_all, 'all_of_us_acs_dist.csv', '/output/')
write_csv(acs_analysis, 'analysis_acs_dist.csv', '/output/')

### Get access rates by race

In [ ]:
# all 
data %>% summarise(hi=mean(access), low=mean(low_access), 
                                      delayed=mean(delayed_ind), afford=mean(afford_ind))

In [ ]:
# access rates by race 
bl<-data %>% group_by(black) %>% summarise(hi=mean(access), low=mean(low_access), 
                                      delayed=mean(delayed_ind), afford=mean(afford_ind))
hisp<-data %>% group_by(is_latino) %>% summarise(hi=mean(access), low=mean(low_access), 
                                      delayed=mean(delayed_ind), afford=mean(afford_ind))
white<-data %>% group_by(white) %>% summarise(hi=mean(access), low=mean(low_access), 
                                      delayed=mean(delayed_ind), afford=mean(afford_ind))

bl
hisp
white

In [ ]:
bl<-bl[bl$black == 1,]
bl$group<-'black'
hisp<-hisp[hisp$is_latino == 1,]
hisp$group<-'hispanic'
white<-white[white$white == 1,]
white$group<-'white'

all<-bind_rows(bl, hisp, white)

In [ ]:
head(all)
access_vars<-c('hi','low','delayed','afford')
long <- all[,c('group',access_vars)] %>%
  pivot_longer(cols = access_vars, 
               names_to = "access_group", 
               values_to = "value")

In [ ]:
 # add on p-value for comparing age, female, black, and hispanic
t.test(AGE ~ noaccess, data=data)

In [ ]:
chisq.test(table(data$fem, data$noaccess))

In [ ]:
chisq.test(table(data$black, data$noaccess))

In [ ]:
chisq.test(table(data$is_latino, data$noaccess))

In [ ]:
# sample sizes for everything
dim(data)

In [ ]:
access<-load_data('access_byperson.csv', "/survey/")
dim(access)

# Summarize task sample

In [ ]:
get_summary<-function(df, label){
    summ<-df %>% summarize(delayed=mean(delayed_ind),
                   afford=mean(afford_ind),
                   hc_int_1yr=mean(hc_int_1_yr),
                   hc_int_2yr=mean(hc_int_2_yr),
                   age=mean(age),
                   male=mean(male),
                   fem=mean(fem),
                   gender_other=mean(gender_other),
                   white=mean(white),
                   black=mean(black),
                   asian=mean(asian),
                   ai_an=mean(ai_an),
                   race_unknown=mean(race_unknown),
                   race_other=mean(race_other),
                   is_latino=mean(is_latino),
                   not_hispanic=mean(not_hispanic),
                   eth_unknown=mean(ethnicity_unknown),
                   ins_uninsured = mean(ins_uninsured),
                   ins_medicare = mean(ins_medicare),
                   ins_medicaid = mean(ins_medicaid),
                   ins_employ = mean(ins_employ),
                   ins_unknown = mean(ins_unknown),
                   ins_other = mean(ins_other), 
                   inc_10 = mean(inc_10),
                   inc_10_49 = mean(inc_10_49),
                   inc_50_99 = mean(inc_50_99),
                   inc_100_199 = mean(inc_100_199),
                   inc_200 = mean(inc_200),
                   inc_unknown = mean(inc_unknown),
                   n=n(),
                   y=mean(y))
    
    summ$group<-label
    return(summ)
}

# group by and summarize -- overall, high access, low access (AND), delay, afford
get_group_summary<-function(df, label, groupvar){
    summ<-df %>% group_by({{ groupvar }}) %>% summarize(delayed=mean(delayed_ind),
                  afford=mean(afford_ind),
                   hc_int_1yr=mean(hc_int_1_yr),
                   hc_int_2yr=mean(hc_int_2_yr),
                   age=mean(age),
                   male=mean(male),
                   fem=mean(fem),
                   gender_other=mean(gender_other),
                   white=mean(white),
                   black=mean(black),
                   asian=mean(asian),
                   ai_an=mean(ai_an),
                   race_unknown=mean(race_unknown),
                   race_other=mean(race_other),
                   is_latino=mean(is_latino),
                   not_hispanic=mean(not_hispanic),
                   eth_unknown=mean(ethnicity_unknown),
                   ins_uninsured = mean(ins_uninsured),
                   ins_medicare = mean(ins_medicare),
                   ins_medicaid = mean(ins_medicaid),
                   ins_employ = mean(ins_employ),
                   ins_unknown = mean(ins_unknown),
                   ins_other = mean(ins_other), 
                   inc_10 = mean(inc_10),
                   inc_10_49 = mean(inc_10_49),
                   inc_50_99 = mean(inc_50_99),
                   inc_100_199 = mean(inc_100_199),
                   inc_200 = mean(inc_200),
                   inc_unknown = mean(inc_unknown),
                   n=n(),
                   y=mean(y))
    
    summ$group<-label
    return(summ)
}

## Diabetes

In [ ]:
diab<-load_data('prediction_data_updt.csv', "/diabetes/")

In [ ]:
keepvars<-c('person_id','y')
diab<-diab[,keepvars]

data_updt<-merge(data, diab, by='person_id')

overall<-get_summary(data_updt, 'overall')

data_updt$access<-ifelse(data_updt$afford_ind == 0 & data_updt$delayed_ind == 0, 1, 0)
high_access<-get_summary(data_updt[data_updt$access == 1,], 'High Access')

data_updt$low_access<-ifelse(data_updt$afford_ind == 1 & data_updt$delayed_ind == 1, 1, 0)
low_access<-get_summary(data_updt[data_updt$low_access == 1,], 'Low Access')

# afford
afford<-get_summary(data_updt[data_updt$afford_ind == 1,], 'Affordability')

# delay
delay<-get_summary(data_updt[data_updt$delayed_ind == 1,], 'Delay')

all<-rbind(overall, high_access, low_access, afford, delay)

write_csv(all, 'diabetes_sample_char.csv', "/output/")

# Self-Reported Conditions

In [ ]:
# pull in self-reported data and merge with the sample frame 
self_tmp<-load_data('self_report_wide.csv', "/survey/")
self<-merge(self_tmp, data, by='person_id')

## Age-adjusted comparison of access groups

In [ ]:
vars<-names(self)
self_vars<-vars[grepl("self_", vars)]
self_vars<-self_vars[!(self_vars %in% c('self_survey_date','emp_self_employed',
                                        'self_reported_category_concept_id','self_reported_category'))]

In [ ]:
self$noaccess<-ifelse(self$afford_ind==1 & self$delayed_ind==1, 1, 0)

In [ ]:
# create 3 groups - high access, low access, other
self$access_groups_delay<-ifelse(self$afford_ind == 0 & self$delayed_ind == 0, 'High Access', 
                           ifelse(self$delayed_ind == 1, 'Delayed Care', 'Other'))

self$access_groups_afford<-ifelse(self$afford_ind == 0 & self$delayed_ind == 0, 'High Access', 
                           ifelse(self$afford_ind == 1, 'Affordability', 'Other'))

In [ ]:
# calculate age weights
self$age_round<-round(self$age,-1)
age_weights<-self %>% group_by(age_round) %>% summarize(weight = n() / nrow(self))

# get counts for each condition 
counts<-colSums(self[,self_vars])
counts_ord <- counts[order(counts, decreasing = TRUE)]

In [ ]:
# create a function to do this by condition 
get_age_adj_prev<-function(df, cond){

    # find prevalence within age groups
    byage<-df %>% group_by(age_round, noaccess) %>% summarise(n=n(), 
                                                              prev=mean(.data[[cond]]),var=var(.data[[cond]]))
    # merge in weights 
    byage<-merge(byage, age_weights, by='age_round')
    
    # age adj
    ageadj<-byage %>% group_by(noaccess) %>% 
        summarize(n=sum(n), prev_wgt = sum(prev*weight), prev_var = sum(prev*weight^2)) %>% 
        mutate(prev_sd = sqrt(prev_var/n))
    
    # run age and gender-adjusted regression and get p-value
    df$y<-df[[cond]]
    
    reg<-glm(y ~ noaccess + age, data=df, family=binomial)
    pval<-summary(reg)$coefficients["noaccess", "Pr(>|z|)"]
    
  
    mean_standard <- round(ageadj[ageadj$noaccess == 0, 'prev_wgt'],3)
    mean_low_access <- round(ageadj[ageadj$noaccess == 1, 'prev_wgt'],3)
    
    res<-as.data.frame(c(cond=cond, standard=mean_standard, low_access = mean_low_access, pval=pval))
    return(res)
}

In [ ]:
# go through top 10 conditions and build dataset 
options(scipen = 999)
top10<-names(counts_ord[1:10])
adj_results<-data.frame()
for (c in top10){
    ret<-get_age_adj_prev(self, c)
    adj_results<-rbind(adj_results, as.data.frame(ret))
}

# use BH correction 
adj_results$pval_adj<-p.adjust(adj_results$pval, method="BH")
options(digits = 7)
adj_results$pval<-round(adj_results$pval, 7)
adj_results$pval_adj<-round(adj_results$pval_adj, 7)
adj_results
write_csv(adj_results, 'age_adj_results.csv', '/output/')